In [ ]:
import pandas as pd

In [ ]:
engine_df_train = pd.read_csv("train_FD001_with_RUL.csv")
engine_df_test= pd.read_csv("test_FD001.csv")

In [ ]:
engine_df_train.head()

,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [ ]:
engine_df_train.shape

(20631, 27)

In [ ]:
engine_df_train = engine_df_train.drop(columns=["unit_number"], errors="ignore")

In [ ]:
engine_df_train.isnull().sum().sum()

np.int64(0)

In [ ]:
constant_cols = [c for c in engine_df_train.columns if "sensor" in c and engine_df_train[c].nunique() == 1]
engine_df_train = engine_df_train.drop(columns=constant_cols)

In [ ]:
engine_df_train.head()

,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_2,sensor_3,sensor_4,sensor_6,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_17,sensor_20,sensor_21,RUL
0,1,-0.0007,-0.0004,100.0,641.82,1589.70,1400.60,21.61,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191
1,2,0.0019,-0.0003,100.0,642.15,1591.82,1403.14,21.61,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190
2,3,-0.0043,0.0003,100.0,642.35,1587.99,1404.20,21.61,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189
3,4,0.0007,0.0000,100.0,642.35,1582.79,1401.87,21.61,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188
4,5,-0.0019,-0.0002,100.0,642.37,1582.85,1406.22,21.61,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = engine_df_train.drop(columns=["RUL"])
y = engine_df_train["RUL"]
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.0 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
import optuna
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

def objective(trial):
    model_name = trial.suggest_categorical(
        "model",
        ["RandomForest", "ExtraTrees", "XGBoost", "LightGBM", "GradientBoosting"]
    )

    if model_name == "RandomForest":
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("rf_n", 300, 1000),
            max_depth=trial.suggest_int("rf_depth", 6, 24),
            min_samples_split=trial.suggest_int("rf_mss", 2, 12),
            min_samples_leaf=trial.suggest_int("rf_msl", 1, 6),
            n_jobs=-1, random_state=42
        )

    elif model_name == "ExtraTrees":
        model = ExtraTreesRegressor(
            n_estimators=trial.suggest_int("et_n", 400, 1400),
            max_depth=trial.suggest_int("et_depth", 6, 28),
            min_samples_split=trial.suggest_int("et_mss", 2, 12),
            min_samples_leaf=trial.suggest_int("et_msl", 1, 6),
            bootstrap=trial.suggest_categorical("et_boot", [False, True]),
            n_jobs=-1, random_state=42
        )

    elif model_name == "XGBoost":
        model = XGBRegressor(
            n_estimators=trial.suggest_int("xgb_n", 400, 2000),
            max_depth=trial.suggest_int("xgb_depth", 3, 10),
            learning_rate=trial.suggest_float("xgb_lr", 1e-3, 0.3, log=True),
            subsample=trial.suggest_float("xgb_sub", 0.6, 1.0),
            colsample_bytree=trial.suggest_float("xgb_col", 0.6, 1.0),
            reg_alpha=trial.suggest_float("xgb_alpha", 0.0, 1.0),
            reg_lambda=trial.suggest_float("xgb_lambda", 0.1, 2.0),
            tree_method="hist", random_state=42, n_jobs=-1
        )

    elif model_name == "LightGBM":
        model = LGBMRegressor(
            n_estimators=trial.suggest_int("lgb_n", 400, 2000),
            max_depth=trial.suggest_int("lgb_depth", -1, 18),
            num_leaves=trial.suggest_int("lgb_leaves", 31, 255),
            learning_rate=trial.suggest_float("lgb_lr", 1e-3, 0.3, log=True),
            subsample=trial.suggest_float("lgb_sub", 0.6, 1.0),
            colsample_bytree=trial.suggest_float("lgb_col", 0.6, 1.0),
            reg_alpha=trial.suggest_float("lgb_alpha", 0.0, 1.0),
            reg_lambda=trial.suggest_float("lgb_lambda", 0.1, 2.0),
            random_state=42, n_jobs=-1
        )

    else:  # GradientBoosting
        model = GradientBoostingRegressor(
            n_estimators=trial.suggest_int("gbr_n", 300, 1200),
            max_depth=trial.suggest_int("gbr_depth", 2, 6),
            learning_rate=trial.suggest_float("gbr_lr", 0.01, 0.3, log=True),
            subsample=trial.suggest_float("gbr_sub", 0.6, 1.0),
            random_state=42
        )


    if isinstance(model, (XGBRegressor, LGBMRegressor)):
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
        )
    else:
        model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    return mae

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Best MAE:", study.best_value)
print("Best params:", study.best_trial.params)

[I 2025-10-09 18:00:37,355] A new study created in memory with name: no-name-f6f28dc2-1d6c-4cfb-b0d1-ad8c0caab209


  0%|          | 0/40 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
train_df = pd.read_csv("train_FD001_with_RUL.csv")
assert "RUL" in train_df.columns, "Your train CSV must include an RUL column."

In [ ]:
drop_cols = ["RUL","unit_number","dataset_type"]
features = [c for c in train_df.columns if c not in drop_cols]
X_full = train_df[features]
y_full = train_df["RUL"]

In [ ]:
scaler = MinMaxScaler()
X_full_scaled = pd.DataFrame(scaler.fit_transform(X_full), columns=features)

In [ ]:
model = ExtraTreesRegressor(
    n_estimators=600, max_depth=22,
    min_samples_split=5, min_samples_leaf=5,
    bootstrap=False, n_jobs=-1, random_state=42
)
model.fit(X_full_scaled, y_full)
print("✅ retrained on full train_FD001")

✅ retrained on full train_FD001


In [ ]:
test_df = pd.read_csv("test_FD001.csv")
rul_true = pd.read_csv("RUL_FD001.csv")["RUL"].values

In [ ]:
last_idx = test_df.groupby("unit_number")["time_in_cycles"].idxmax()
test_last = test_df.loc[last_idx].sort_values("unit_number").reset_index(drop=True)


feat_in_both = [c for c in features if c in test_last.columns]
missing = [c for c in features if c not in feat_in_both]
if missing:
    raise ValueError(f"Missing in test: {missing}. Recreate the same engineered features for test before evaluating.")
X_test_scaled = pd.DataFrame(scaler.transform(test_last[feat_in_both]), columns=feat_in_both)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np

pred = model.predict(X_test_scaled)

mae  = mean_absolute_error(rul_true, pred)
rmse = np.sqrt(mean_squared_error(rul_true, pred))
r2   = r2_score(rul_true, pred)
acc10 = np.mean(np.abs(rul_true - pred) <= 10) * 100


print(f"MAE : {mae:.2f} cycles")
print(f"RMSE: {rmse:.2f} cycles")
print(f"R²  : {r2:.3f}")
print(f"Accuracy (±10 cycles): {acc10:.1f}%")

MAE : 18.97 cycles
RMSE: 25.25 cycles
R²  : 0.631
Accuracy (±10 cycles): 42.0%


In [ ]:
import joblib

out = test_last[["unit_number","time_in_cycles"]].copy()
out["pred_RUL"] = pred
out.to_csv("FD001_test_predictions.csv", index=False)

joblib.dump(model, "best_model_fd001.joblib")
joblib.dump(scaler, "scaler_fd001.joblib")
print("saved model")

saved model


In [ ]:
files.download("best_model_fd001.joblib")
files.download("scaler_fd001.joblib")
files.download("FD001_test_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train = pd.read_csv("train_FD001_with_RUL.csv")
train.to_excel("train_FD001_with_RUL.xlsx", index=False)

In [ ]:
files.download("train_FD001_with_RUL.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>